#### colab로 시작하기

<a href="https://colab.research.google.com/github/LDJWJ/AISTUDY/blob/master/DL05_02_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 02 RNN을 이용한 단어 자동 완성

* 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한글자를 추천하여 단어를 완성한다.
* dynamic_rnn의 sequence_length 옵션을 사용하면 가변 길이의 단어를 학습 시킬 수 있다.
* 학습시킬 데이터는 영문자로 구성된 임의의 단어를 사용함.
* 한 글자 한글자가 한 단계의 입력값, **총 글자 수가 전체 단계**가 된다.
* word : 4글자, w, o, r, d - 총 4단계

In [2]:
import tensorflow as tf
import numpy as np

### 알파벳 
* 각각의 알파벳에 대해 index를 구한다.

In [3]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

In [4]:
# one-hot 인코딩 사용 및 디코딩을 하기 위해 연관 배열을 만듭니다.
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)
print(dic_len)

26


### 순서를 갖는 데이터 준비
 * 네글자의 단어를 준비
 * 앞의 세글자는 입력(X)로 하고, 
 * 네번째 글자는 출력(Y)로 구분할 것임.

In [5]:
# wor -> X, d -> Y
# woo -> X, d -> Y

In [6]:
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 
            'load', 'love', 'kiss', 'kind']

### 단어들을 전달받아, 데이터 전처리를 수행
  * 앞의 세글자 : 원핫 인코딩 수행하여 반환
  * 마지막 글자 : 알파벳의 인덱스 숫자로 반환

In [7]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    # seq_data : 전체 단어들
    # seq : 한 단어
    # input : 단어의 마지막 글자를 제외한 글자들의 index(알파벳)
    # target : 한 단어의 마지막 글자(인덱스 반환)
    for seq in seq_data:
        # 여기서 생성하는 input_batch 와 target_batch 는
        # 알파벳 배열의 인덱스 번호 입니다.
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
        input = [num_dic[n] for n in seq[:-1]]
        # 3, 3, 15, 4, 3 ...
        
        target = num_dic[seq[-1]]  # 마지막 글자 인덱스
        
        # input의 값을 원핫 인코딩을 수행
        # if input is [0, 1, 2]:
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        
        input_batch.append(np.eye(dic_len)[input])
        
        # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
        # label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
        # 이 예제에서 사용할 손실 함수인 
        # sparse_softmax_cross_entropy_with_logits 는 원핫 인코딩을 사용하지 않음.
        # index 를 그냥 넘겨주면 됩니다.
        target_batch.append(target)

    return input_batch, target_batch

In [8]:
make_batch(['word'])

([array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])], [3])

### 데이터를 전처리
```
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 
            'load', 'love', 'kiss', 'kind']
```
 * 위의 단어들 각각 3글자를 인덱스로 변환 후, 원핫 인코딩
 * 위의 단어들 각각 마지막 글자를 인덱스로 반환

In [9]:
# 전체 단어의 데이터 셋을 a(원핫 인코딩), b(숫자)
# RNN 모델에 맞춰 데이터 변경.

## word
a, b= make_batch(seq_data)
print(a[0])  # w, o, r - 원핫 인코딩
print(b[0])  # d - 숫자로 
print(b)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]]
3
[3, 3, 15, 4, 3, 11, 3, 4, 18, 3]


### 하이퍼 파라미터 설정
 * 학습율, 은닉층 노드, 총 에폭수
 * RNN의 step은 입력 단어의 글자수이므로 3
 * 입력값은 원핫 인코딩되어 입력(알파벳의 개수 26) - n_input, n_class

In [10]:
learning_rate = 0.01  # 학습률
n_hidden = 128        # 은닉층 노드수 
total_epoch = 30      # 총 에폭수 
n_step = 3            # RNN 스텝

# 입력값 크기. 알파벳에 대한 one-hot 인코딩이므로 26개가 됩니다.
# 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
# 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류합니다.
n_input = n_class = dic_len

### 신경망 모델 구성
 * X, Y 변수 및 가중치 지정.
 * RNN 셀 생성(cell1, cell2) 및 멀티셀 생성(LSTM 셀)
 * tf.nn.dynamic_rnn 함수를 활용하여 심층 순환 신경망 생성
 * 손실함수 및 최적화 함수
 * 학습

In [11]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])

# 비용함수에 sparse_softmax_cross_entropy_with_logits 을 사용하므로
# 출력값과의 계산을 위한 원본값의 형태는 one-hot vector가 아니라 인덱스 숫자를 그대로 사용하기 때문에
# 다음처럼 하나의 값만 있는 1차원 배열을 입력값으로 받습니다.
# [3] [3] [15] [4] ...
# 기존처럼 one-hot 인코딩을 사용한다면 입력값의 형태는 [None, n_class] 여야합니다.
Y = tf.placeholder(tf.int32, [None])

# 가중치와 Bias
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

### RNN 셀을 생성
* LSTM은 BasicRNNCell가 아닌 **BasicLSTMCell**의 함수를 사용
* DropoutWrapper 함수 : RNN에도 DropoutWrapper 함수로 Dropout 기법 적용이 가능하다. 

In [16]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # RNN 셀을 생성
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5) # 과적합 방지를 위한 Dropout 기법

cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성
print(cell1)
print(cell2)

### 여러 셀을 조합

In [17]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
print(multi_cell)

### tf.nn.dynamic_rnn 함수를 이용해 순환 신경망

In [19]:
# time_major=True
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
print(outputs)
print()
print(states)

Tensor("rnn_1/transpose_1:0", shape=(?, 3, 128), dtype=float32)

(LSTMStateTuple(c=<tf.Tensor 'rnn_1/while/Exit_3:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn_1/while/Exit_4:0' shape=(?, 128) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn_1/while/Exit_5:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn_1/while/Exit_6:0' shape=(?, 128) dtype=float32>))


## 손실함수, 최적화 함수 적용
 * 앞의 것과 동일한 방식으로 하여 출력층을 만든다.
 * 순서 바꾸기
   * [batch_size, n_step, n_hidden] -> [n_step, batch_size, n_hidden]
   * -> [batch_size, n_hidden]
     * shape=(3, ?, 128)  -> (?, 128)
     * (?, 128) * (128, 26) -> (?, 26)

In [20]:
# 최종 결과는 one-hot 인코딩 형식으로 만듭니다
# 
outputs1 = tf.transpose(outputs, [1, 0, 2])
outputs1 = outputs1[-1]
model = tf.matmul(outputs1, W) + b
print(model)
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Tensor("add:0", shape=(?, 26), dtype=float32)


### 신경망 실행 - 그래프 실행

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

In [22]:
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 2.839599
Epoch: 0002 cost = 2.060616
Epoch: 0003 cost = 1.404584
Epoch: 0004 cost = 0.888910
Epoch: 0005 cost = 0.911129
Epoch: 0006 cost = 0.457103
Epoch: 0007 cost = 0.498594
Epoch: 0008 cost = 0.397425
Epoch: 0009 cost = 0.290067
Epoch: 0010 cost = 0.380558
Epoch: 0011 cost = 0.289656
Epoch: 0012 cost = 0.145673
Epoch: 0013 cost = 0.155696
Epoch: 0014 cost = 0.177595
Epoch: 0015 cost = 0.150806
Epoch: 0016 cost = 0.073050
Epoch: 0017 cost = 0.311992
Epoch: 0018 cost = 0.106288
Epoch: 0019 cost = 0.028718
Epoch: 0020 cost = 0.063455
Epoch: 0021 cost = 0.097836
Epoch: 0022 cost = 0.029299
Epoch: 0023 cost = 0.015738
Epoch: 0024 cost = 0.021585
Epoch: 0025 cost = 0.269463
Epoch: 0026 cost = 0.024357
Epoch: 0027 cost = 0.009378
Epoch: 0028 cost = 0.087993
Epoch: 0029 cost = 0.051703
Epoch: 0030 cost = 0.025511
최적화 완료!


### 결과 확인

In [23]:
# 레이블값이 정수이므로 예측값도 정수로 변경해줍니다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)

# one-hot 인코딩이 아니므로 입력값을 그대로 비교합니다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

In [24]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
